In [1]:
# importing the libraries 
#!pip install dask
!pip install pyarrow
import dask
import dask.dataframe as dd
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import os
import glob
from dask.diagnostics import ProgressBar 

# 1. Dataset:
This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to 2017. 

# 2. Variables of the dataset:

ID - Unique identifier for the record.

Case Number - The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.

Date - Date when the incident occurred. this is sometimes a best estimate.

Block - The partially redacted address where the incident occurred, placing it on the same block as the actual address.

IUCR - The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.

Primary Type - The primary description of the IUCR code.

Description - The secondary description of the IUCR code, a subcategory of the primary description.

Location Description - Description of the location where the incident occurred.

Arrest - Indicates whether an arrest was made.

Domestic - Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act.

Beat - Indicates the beat where the incident occurred. A beat is the smallest police geographic area – each beat has a dedicated police beat car. Three to five beats make up a police sector, and three sectors make up a police district. The Chicago Police Department has 22 police districts. See the beats at https://data.cityofchicago.org/d/aerh-rz74.

District - Indicates the police district where the incident occurred. See the districts at https://data.cityofchicago.org/d/fthy-xz3r.

Ward - The ward (City Council district) where the incident occurred. See the wards at https://data.cityofchicago.org/d/sp34-6z76.

Community Area - Indicates the community area where the incident occurred. Chicago has 77 community areas. See the community areas at https://data.cityofchicago.org/d/cauq-8yn6.

FBI Code - Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS). See the Chicago Police Department listing of these classifications at http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html.

X Coordinate - The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.

Y Coordinate - The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.

Year - Year the incident occurred.

Updated On - Date and time the record was last updated.

Latitude - The latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.

Longitude - The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.

Location - The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block.

# 3. Load data files:

In [2]:
#Loading the four CSV files
from glob import glob
filenames=sorted(glob(os.path.join("Data","*.csv")))
filenames

['Data\\Chicago_Crimes_2001_to_2004.csv',
 'Data\\Chicago_Crimes_2005_to_2007.csv',
 'Data\\Chicago_Crimes_2008_to_2011.csv',
 'Data\\Chicago_Crimes_2012_to_2017.csv']

In [3]:
dask_all_file=dd.read_csv(filenames,error_bad_lines=False,assume_missing=True,dtype={'ID': np.int64,'PrimaryType': 'str','FBI Code': 'object','Beat': np.uint16,'IUCR': 'str','Latitude': 'object',
'Y Coordinate': 'object','Location': 'str'},  parse_dates=['Date'],infer_datetime_format=True)

# 4. Overview of dataset:

In [4]:
dask_all_file

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
npartitions=33,,,,,,,,,,,,,,,,,,,,,,,
,float64,int64,object,datetime64[ns],object,object,object,object,object,bool,bool,uint16,float64,float64,float64,object,float64,object,float64,object,object,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [5]:
dask_all_file.dtypes

Unnamed: 0                     float64
ID                               int64
Case Number                     object
Date                    datetime64[ns]
Block                           object
IUCR                            object
Primary Type                    object
Description                     object
Location Description            object
Arrest                            bool
Domestic                          bool
Beat                            uint16
District                       float64
Ward                           float64
Community Area                 float64
FBI Code                        object
X Coordinate                   float64
Y Coordinate                    object
Year                           float64
Updated On                      object
Latitude                        object
Longitude                      float64
Location                        object
dtype: object

In [6]:
dask_all_file.head()

/home/kuntalac/miniconda3/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  df = pandas_read_text(


,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,879.0,4786321,HM399414,2004-01-01 00:01:00,082XX S COLES AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,...,7.0,46.0,06,NaN,NaN,2004.0,08/17/2015 03:03:40 PM,NaN,NaN,NaN
1,2544.0,4676906,HM278933,2003-03-01 00:00:00,004XX W 42ND PL,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,...,11.0,61.0,26,1173974.0,1876757.0,2003.0,04/15/2016 08:55:02 AM,41.817229156,-87.637328,"(41.817229156, -87.637328162)"
2,2919.0,4789749,HM402220,2004-06-20 11:00:00,025XX N KIMBALL AVE,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,...,35.0,22.0,20,NaN,NaN,2004.0,08/17/2015 03:03:40 PM,NaN,NaN,NaN
3,2927.0,4789765,HM402058,2004-12-30 20:00:00,045XX W MONTANA ST,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,...,31.0,20.0,06,NaN,NaN,2004.0,08/17/2015 03:03:40 PM,NaN,NaN,NaN
4,3302.0,4677901,HM275615,2003-05-01 01:00:00,111XX S NORMAL AVE,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,...,34.0,49.0,06,1174948.0,1831051.0,2003.0,04/15/2016 08:55:02 AM,41.691784636,-87.635116,"(41.691784636, -87.635115968)"


# 5. Data Cleaning:

## 5.1 Drop unwanted columns:

**We found the following columns not useful for our data analysis, so we decided to drop them.**

In [6]:
columns_to_drop = ['Unnamed: 0','ID','Case Number','IUCR','X Coordinate','Y Coordinate','Updated On','Location' ]
dask_all_file1 = dask_all_file.drop(columns_to_drop, axis=1)

In [7]:
dask_all_file1.columns

Index(['Date', 'Block', 'Primary Type', 'Description', 'Location Description',
       'Arrest', 'Domestic', 'Beat', 'District', 'Ward', 'Community Area',
       'FBI Code', 'Year', 'Latitude', 'Longitude'],
      dtype='object')

## 5.2 Persist DataFrame in memory:

**Persist is important because Dask DataFrame is lazy by default. It is often ideal to load, filter, and shuffle data once and keep this result in memory. Afterwards, each of the several complex queries can be based off of this in-memory data rather than have to repeat the full load-filter-shuffle process each time.**

*Source: "https://docs.dask.org/en/stable/dataframe-best-practices.html"*

In [8]:
# Persist the dataframe in memory
dask_all_file1 = dask_all_file1.persist()

b'Skipping line 210938: expected 23 fields, saw 24\n'
b'Skipping line 24743: expected 23 fields, saw 24\n'
b'Skipping line 132222: expected 23 fields, saw 41\n'


## 5.3 Handle missing values:

In [9]:
missing_values = dask_all_file1.isnull().sum()
missing_values

Dask Series Structure:
npartitions=1
Arrest    int64
Year        ...
dtype: int64
Dask Name: dataframe-sum-agg, 100 tasks

In [10]:
missing_count = ((missing_values / dask_all_file1.index.size) * 100)
missing_count

Dask Series Structure:
npartitions=1
Arrest    float64
Year          ...
dtype: float64
Dask Name: mul, 169 tasks

In [11]:
%%time
with ProgressBar(): 
#ProgressBar is a function provided out of the dask.diagnostics package which will display a progress bar in your terminal to let you know how far a long a particular task is while it’s executing.
    missing_count_pct = missing_count.compute()
missing_count_pct

[########################################] | 100% Completed | 12.3s
Wall time: 12.4 s


Date                    0.000000
Block                   0.000000
Primary Type            0.000000
Description             0.000000
Location Description    0.025059
Arrest                  0.000000
Domestic                0.000000
Beat                    0.000000
District                0.001146
Ward                    8.817518
Community Area          8.841028
FBI Code                0.000000
Year                    0.000000
Latitude                1.329420
Longitude               1.329433
dtype: float64

## 5.4 Drop rows with missing values:

**As we have around 7 million of records and percentage of missing in each column is extremely low (less than 10%), we decided to drop the missing instead of updating which is very expensive operation.**

In [12]:
%%time
rows_to_drop = list(missing_count_pct[(missing_count_pct > 0) & (missing_count_pct < 5)].index)
dask_all_file1 = dask_all_file1.dropna(subset=rows_to_drop)    

Wall time: 110 ms


In [13]:
dask_all_file1.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 15 entries, Date to Longitude
dtypes: datetime64[ns](1), object(6), bool(2), float64(5), uint16(1)

## 5.5 Handle Duplicates

### 5.5.1 Drop duplicate records

In [14]:
%%time
dask_all_file1 = dask_all_file1.drop_duplicates()

Wall time: 99.9 ms


In [15]:
%%time
with ProgressBar():
    dask_all_file1 = dask_all_file1.set_index('Date')

[########################################] | 100% Completed |  1min 22.9s
Wall time: 1min 23s


# 6. Writing to Parquet file and Loading DataFrame from Parquet file

**Dask DataFrame users are encouraged to store and load data using Parquet. Apache Parquet is a columnar binary format that is easy to split into multiple files (easier for parallel loading) and is generally much simpler to deal with.**

Source: https://docs.dask.org/en/stable/dataframe-best-practices.html

In [16]:
%%time
with ProgressBar():
    dask_all_file1.to_parquet('crimes-2001-2017.snappy.parq', compression='SNAPPY')

[########################################] | 100% Completed |  1min 56.0s
Wall time: 1min 57s


In [18]:
%%time
crimes_df = dd.read_parquet('crimes-2001-2017.snappy.parq', index='Date')

Wall time: 250 ms


In [19]:
%%time
crimes_df = crimes_df.persist()

Wall time: 1min 3s


In [20]:
# dataframe structure
crimes_df

,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Year,Latitude,Longitude
npartitions=1,,,,,,,,,,,,,,
2001-01-01 00:00:00,object,object,object,object,bool,bool,uint16,float64,float64,float64,object,float64,object,float64
2017-01-18 10:12:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [21]:
%%time
crimes_df.head()

Wall time: 152 ms


,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Year,Latitude,Longitude
Date,,,,,,,,,,,,,,
2001-01-01,089XX S BRANDON AVE,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,424,4.0,10.0,46.0,06,2001.0,41.733539408,-87.547156
2001-01-01,027XX W NELSON ST,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,1411,14.0,NaN,NaN,14,2001.0,41.936549915,-87.697098
2001-01-01,033XX W 64TH ST,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,823,8.0,15.0,66.0,06,2001.0,41.777203808,-87.705942
2001-01-01,057XX S JUSTINE ST,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,False,713,7.0,16.0,67.0,06,2001.0,41.789165627,-87.663084
2001-01-01,039XX S CALIFORNIA AVE,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,912,9.0,12.0,58.0,20,2001.0,41.822690348,-87.694491


# 7. Data Analysis

## 7.1 Single Column analysis

### 7.1.1 Column Year: Years with most criminal records

In [22]:
%%time
with ProgressBar():
    crime_by_year=crimes_df.groupby([crimes_df.index.year]).size().compute()
        
top_10_years = crime_by_year.nlargest(10).sort_values(axis=0,ascending=False)
top_10_years

[########################################] | 100% Completed |  2.0s
Wall time: 2.17 s


Date
2001    481676
2002    469854
2003    468972
2005    448245
2006    443550
2007    434313
2008    419149
2009    385372
2004    383414
2010    367997
dtype: int64

### 7.1.2. Column Primary Type: type of criminal action, which one was the most common

In [39]:
%%time
with ProgressBar():
    crime_count_by_type = crimes_df.groupby('Primary Type').size().compute()
 
top_10_crime_types = crime_count_by_type.nlargest(10).sort_values(axis=0,ascending=False)
top_10_crime_types

[########################################] | 100% Completed |  0.7s
Wall time: 761 ms


Primary Type
THEFT                  1262435
BATTERY                1114872
CRIMINAL DAMAGE         701857
NARCOTICS               656837
OTHER OFFENSE           376185
ASSAULT                 373118
BURGLARY                357618
MOTOR VEHICLE THEFT     286417
ROBBERY                 230824
DECEPTIVE PRACTICE      211106
dtype: int64

### 7.1.3. Column Domestic: Rate of domestic criminal offenses vs non-domestic ones

### 7.1.4. Column Arrest: Rate of criminal acts involving arrest

In [31]:
#crimes_df[['Arrest','Primary Type']].groupby(crimes_df[crimes_df['Arrest']==True]).size().compute()

### 7.1.5. Columns Latitude, longitude: To add them on a map to determine which areas concentrate most of the criminality

In [ ]:
# this can be in visualization

### 7.1.6. Column District: Top 5 of most violent districts

In [33]:
# upload the community file
community_areas = pd.read_csv('community\\chicago-community-areas.csv')

# apply groupby function and calculate the amount of crime, rename to Total
community_areas['Total'] = crimes_df.groupby('Community Area').size().compute().rename('Total')

# sort value
community_crime = community_areas.sort_values(by='Total', ascending=False).dropna()

# drop unused columns and reindex
community_crime = community_crime.drop(['CommunityArea','Side'], axis=1) 
community_crime = community_crime.set_index('CommunityName')

# select top 5 of most violent districts
community_crime.nlargest(5,'Total')

,Total
CommunityName,
Austin,350737
Near North Side,179280
South Shore,179036
Humboldt Park,173107
West Englewood,162558


..

## 7.2 Analysis involving more than two categories

### 7.2.1 Column Primary Type and Year: Year evolution of the top 5 most common criminal actions

In [47]:
top_5_crime_types = crime_count_by_type.nlargest(5).sort_values(axis=0,ascending=False)
a=top_5_crime_types.index.to_list()
a

['THEFT', 'BATTERY', 'CRIMINAL DAMAGE', 'NARCOTICS', 'OTHER OFFENSE']

In [79]:
top_5_crimes = crimes_df[(crimes_df['Primary Type']=='THEFT') |
                       (crimes_df['Primary Type']=='BATTERY') |
                       (crimes_df['Primary Type']=='CRIMINAL DAMAGE') |
                       (crimes_df['Primary Type']=='NARCOTICS') |
                       (crimes_df['Primary Type']=='OTHER OFFENSE')]

In [80]:
crimes_groupby = top_5_crimes.groupby([pd.Grouper('Year'), 'Primary Type'])
top_crime_grouped = crimes_groupby['Primary Type'].count().compute().rename('Count')


In [81]:
top_crime_grouped

Year    Primary Type   
2001.0  BATTERY            93000
        CRIMINAL DAMAGE    55444
        NARCOTICS          49862
        OTHER OFFENSE      29498
        THEFT              98298
                           ...  
2016.0  BATTERY            48647
        CRIMINAL DAMAGE    30123
        NARCOTICS          10633
        OTHER OFFENSE      15886
        THEFT              58202
Name: Count, Length: 80, dtype: int64

### 7.2.2 Column Districts and Year: How violence evolved thoughout the years in the top 5 most violent districts

In [97]:
top_5_districts = community_crime[(community_crime.index=='Austin') |
                       (community_crime.index=='Near North Side') |
                       (community_crime.index=='South Shore') |
                       (community_crime.index=='Humboldt Park') |
                       (community_crime.index=='West Englewood')]

In [101]:
community_areas['Total'] = crimes_df.groupby('Community Area').size().compute().rename('Total')

# sort value
community_crime = community_areas.sort_values(by='Total', ascending=False).dropna()


In [108]:
community_crime.nlargest(5,columns='Total').CommunityArea.to_list()

[25, 8, 43, 23, 67]

In [121]:
top_5_districts=crimes_df[(crimes_df['Community Area'] == 25.)|
          (crimes_df['Community Area'] == 8.) |
          (crimes_df['Community Area'] == 43.) |
          (crimes_df['Community Area'] == 23.) |
          (crimes_df['Community Area'] == 67.)]

In [124]:
crimes_groupby = top_5_districts.groupby([pd.Grouper('Year'), 'Community Area'])
top_crime_grouped = crimes_groupby['Community Area'].count().compute().rename('Count')


In [125]:
top_crime_grouped

Year    Community Area
2001.0  8.0                  87
        23.0                221
        25.0                263
        43.0                104
        67.0                 78
                          ...  
2016.0  25.0              15690
        43.0               7661
        67.0               6437
2017.0  23.0                  1
        25.0                  8
Name: Count, Length: 82, dtype: int64

## 8. Data visualization:

- Top 10 locations by total crimes on those locations
- Top 10 primary types of crimes and their number of occurence
- Number of cases by type
- Comparing Arrested or Not
- Plot Arrested Rate
- Number of cases per Year vs Months (Can use heatmap and bar) (we can also do a stacked graph x=month, y=total crimes , color of stack representing primary type.)
- Crime count total [2001-2017]  Vs. Month of the year- Which month of the year the crime count is the highest?
      -- one more where we add primary type of crimes also in monthwise counts
- Crime count total [2001-2017]  Vs. Day of the week
- No. of Crime per year-month (Aggregate by Year and Month) - 12 months Rolling sum ? We can prove if the crime rate (or crimes) is increasing or decreasing over these years
- Plot the location on the map, - Most Common 10 Crime Places
- Plot no. of crimes by location - on map, locations where total crimes r higher?
- Mapping Theft Crime Locations

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6ab987db-b709-4ef7-9bee-384a40c4b1c2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>